In [20]:
import sys
!{sys.executable} -m pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 25.4 MB/s eta 0:00:0000:010:01


In [24]:
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import sklearn
from sklearn.model_selection import train_test_split
import math
from collections import Counter


In [2]:
bt_df = pd.read_csv("blood_transfusion.csv")
bt_df.describe()

,months_since_last_donation,total_number_of_donations,total_blood_donated,months_since_first_donation,class
count,748.000000,748.000000,748.000000,748.000000,748.000000
mean,9.506684,5.514706,1378.676471,34.282086,0.237968
std,8.095396,5.839307,1459.826781,24.376714,0.426124
min,0.000000,1.000000,250.000000,2.000000,0.000000
25%,2.750000,2.000000,500.000000,16.000000,0.000000
50%,7.000000,4.000000,1000.000000,28.000000,0.000000
75%,14.000000,7.000000,1750.000000,50.000000,0.000000
max,74.000000,50.000000,12500.000000,98.000000,1.000000


In [ ]:
months_since_last_donation_avg = np.zeros((75, 3))

for i in range(len(months_since_last_donation_avg)):
    months_since_last_donation_avg[i, 0] = i+1

for index, row in bt_df.iterrows():
    months_since_last_donation_avg[int(row[0]), 2] += 1
    avg_class = (months_since_last_donation_avg[int(row[0]), 1] + row[4])/months_since_last_donation_avg[int(row[0]), 2]
    months_since_last_donation_avg[int(row[0]), 1] = avg_class

months_since_last_donation_fig = go.Figure()

# Iterate over each row in the matrix and plot it
months_since_last_donation_fig.add_trace(go.Scatter(
    x=months_since_last_donation_avg[:, 0],  # Use the first column as x-values
    y=months_since_last_donation_avg[:, 1],  # Use the second column as y-values
    mode='lines+markers',  # Connect the points with lines
    name='Data Points'
))

# Update layout for titles and labels
months_since_last_donation_fig.update_layout(
    title='Chances of coming back after last donation',
    xaxis_title='Months passed since last donation',
    yaxis_title='Chances of coming back',
    xaxis=dict(range=[0, 30])
)

# Show the plot
months_since_last_donation_fig.show()

distribution_fig = go.Figure()

for i in range(len(months_since_last_donation_avg)):
    distribution_fig.add_trace(go.Bar(
        x=months_since_last_donation_avg[:, 0],  # Use column names as x-coordinates
        y=months_since_last_donation_avg[:, 2],  # Use row values as y-coordinates
        marker = dict(color = 'red')
    ))

distribution_fig.update_layout(
    xaxis=dict(range=[0, 30])
)

distribution_fig.show()


In [68]:
segments_df= np.array_split(bt_df, 15)

class_distribution_fig = go.Figure()
avg_class = np.zeros((15, 2))

for i in range(len(segments_df)):
    avg_class[i][1] = sum(split_df[i]['class'])/len(segments_df[i])
    avg_class[i][0] = i+1

class_distribution_fig = go.Figure()

class_distribution_fig.add_trace(go.Scatter(
    x=avg_class[:, 0],  # Use the first column as x-values
    y=avg_class[:, 1],  # Use the second column as y-values
    mode='lines+markers',  # Connect the points with lines
    name='Data Points'
))

class_distribution_fig.update_layout(
    title='Distribution of class variable across data',
    xaxis_title='Segments of Data',
    yaxis_title='Average Value of Class',
)

class_distribution_fig.show()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning:

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.



Task 3: Creating a train and test set

In [97]:
class_attribute = bt_df['class']
variables = bt_df.drop(columns = ["class"])

class_train8020, class_test8020, variables_train8020, variables_test8020= train_test_split(class_attribute, variables, test_size=0.2, train_size = 0.8)
class_train9010, class_test9010, variables_train9010, variables_test9010= train_test_split(class_attribute, variables, test_size=0.1, train_size = 0.9)

Task 4: Classification Algorithms

In [120]:
def euclidean_distance(point1, point2):
    distance = 0
    for i in range(len(point1)):
        distance += (point1[i] - point2[i]) ** 2
    return math.sqrt(distance)

def get_neighbors(training_set, labels, test_instance, k):
    distances = []
    for index in range(len(training_set)):
        dist = euclidean_distance(test_instance, training_set[index])
        distances.append((labels[index], dist))
    
    distances.sort(key=lambda x: x[1])
    neighbors = distances[:k]
    return [neighbor[0] for neighbor in neighbors]

def predict_classification(training_set, labels, test_instance, k):
    neighbors = get_neighbors(training_set, labels, test_instance, k)
    count = Counter(neighbors) 
    prediction = count.most_common(1)[0][0]
    return prediction

def precision(predicted, actual):
    predicted = np.array(predicted)
    actual = np.array(actual)

    true_positive = np.sum((predicted == 1) & (actual == 1))
    false_positive = np.sum((predicted == 1) & (actual == 0))

    if (true_positive + false_positive) == 0:
        return 0.0

    precision = true_positive / (true_positive + false_positive)
    return precision


predicted = np.zeros(150)
correct= np.zeros(150)

for i in range(len(predicted)):
    predicted_label = predict_classification(variables_train8020.to_numpy(), class_train8020.to_numpy(), variables_test8020.to_numpy()[i], 20)
    predicted[i] = predicted_label
    correct[i] = class_test8020.to_numpy()[i]
    

evaluater_data = {'Predicted Class': predicted, 'Correct Class': correct}
evaluater = pd.DataFrame(evaluater_data)

print(f'Precision of 80-20 split usiong KNN classifier: {precision(evaluater['Predicted Class'], evaluater['Correct Class'])*100}%')

predicted = np.zeros(75)
correct= np.zeros(75)

for i in range(len(predicted)):
    predicted_label = predict_classification(variables_train9010.to_numpy(), class_train9010.to_numpy(), variables_test9010.to_numpy()[i], 20)
    predicted[i] = predicted_label
    correct[i] = class_test9010.to_numpy()[i]
    

evaluater_data = {'Predicted Class': predicted, 'Correct Class': correct}
evaluater = pd.DataFrame(evaluater_data)

print(f'Precision of 90-10 split usiong KNN classifier: {precision(evaluater['Predicted Class'], evaluater['Correct Class'])*100}%')

Precision of 80-20 split usiong KNN classifier: 40.0%
Precision of 90-10 split usiong KNN classifier: 75.0%
